In [ ]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [ ]:
df = pd.read_csv("../data/test.csv")

In [ ]:
df

## (0) Some minor data preprocessing

In [ ]:
# Fill NAs with 0s and create GDP per capita column
df = df.fillna(0)
df

In [ ]:
df.drop('time', axis = 1, inplace = True)
df.insert(0,'time',df.index.values)

In [ ]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()

In [ ]:
df

## (1) CO2 emission over time by continent

In [ ]:
time_slide_l = (
    pn.widgets.IntSlider(
        name = "Time slider",
        start = 0,
        end = 2723,
        step = 1,
        value = 1
    )
)

In [ ]:
time_slide_r = (
    pn.widgets.IntSlider(
        name = "Time slider",
        start = 0,
        end = 2723,
        step = 1,
        value = 2723
    )
)

In [ ]:
# Radio buttons for CO2 measures
yaxis_shoulder = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['shoulderLangle', 'shoulderRangle',],
    button_type='success'
)

In [ ]:

shoulder_pipeline = (
    idf[
        (idf.time <= time_slide_l &
         idf.time <= time_slide_r) 
    ]
    .reset_index()
    .sort_values(by='time')  
    .reset_index(drop=True)
)

In [ ]:
shoulder_pipeline

In [ ]:
shoulder_plot = shoulder_pipeline.hvplot(x = 'time',  y=yaxis_shoulder,line_width=2, title="Shoulder Plot")
shoulder_plot

In [ ]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Wearable Device for Immersive Virtual Reality Control andApplication in Upper Limbs Motor Rehabilitation dashboard', 
    sidebar=[pn.pane.Markdown(""), 
             pn.pane.Markdown("#### Carbon dioxide emissions are the primary driver of global climate change. It’s widely recognised that to avoid the worst impacts of climate change, the world needs to urgently reduce emissions. But, how this responsibility is shared between regions, countries, and individuals has been an endless point of contention in international discussions."), 
             pn.pane.PNG('device.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),   
             time_slide_l, time_slide_r],
    main=[pn.Row(pn.Column(yaxis_shoulder, 
                           shoulder_plot.panel(width=700), margin=(0,25)), 
                  shoulder_plot.panel(width=700), margin=(0,25)), 
          pn.Row(pn.Column(yaxis_shoulder, 
                           shoulder_plot.panel(width=700), margin=(0,25)), 
                  shoulder_plot.panel(width=700), margin=(0,25))],
    accent_base_color="#87d8b0",
    header_background="#85d8b0",
)
# template.show()
template.servable();